In [165]:
#install.packages('ggplot2')
#install.packages('caTools')
#install.packages("readr")
#install.packages("psych")
#install.packages("fastDummies")

asdasdasda

In [166]:
library(caTools)
library(readr)
library(psych)
library(dplyr)
library(fastDummies)


# 1. DATA ANALYSIS

### 1. Import the data set. Make sure the column names are imported as intended.

In [167]:
data <- read_csv("/Users/gabriel/Documents/GitHub/CausalAI-Course/Labs/Assignment/Assignment_1/wage2015_subsample_inference.csv")
print(data)
spec(data) 

Rows: 5150 Columns: 21
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (21): rownames, wage, lwage, sex, shs, hsg, scl, clg, ad, mw, so, we, ne...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 5,150 x 21
   rownames  wage lwage   sex   shs   hsg   scl   clg    ad    mw    so    we
      <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1       10  9.62  2.26     1     0     0     0     1     0     0     0     0
 2       12 48.1   3.87     0     0     0     0     1     0     0     0     0
 3       15 11.1   2.40     0     0     1     0     0     0     0     0     0
 4       18 13.9   2.63     1     0     0     0     0     1     0     0     0
 5       19 28.8   3.36     1     0     0     0     1     0     0     0     0
 6       30 11.7   2.46     1     0     0     0     1     0     0     0     0
 7       43 19.2   2.96     1     0     1     0     0     0     0     0     0
 8       44 19.2   2.96     0     0     1     0     0     0     0     0     0
 9       47 12     2.48     1     0     1     0     0     0     0     0     0
10       71 19.2   2.96     1     0     0     0     1     0     0     0     0
# i 5,140 more rows
# i 9 more variables:

cols(
  rownames = col_double(),
  wage = col_double(),
  lwage = col_double(),
  sex = col_double(),
  shs = col_double(),
  hsg = col_double(),
  scl = col_double(),
  clg = col_double(),
  ad = col_double(),
  mw = col_double(),
  so = col_double(),
  we = col_double(),
  ne = col_double(),
  exp1 = col_double(),
  exp2 = col_double(),
  exp3 = col_double(),
  exp4 = col_double(),
  occ = col_double(),
  occ2 = col_double(),
  ind = col_double(),
  ind2 = col_double()
)

### 2. Are there missing values? Display the number of missings (if any) of each variable in the data set.

In [168]:
missing_values <- colSums(is.na(data))
missing_df <- data.frame(Variable = names(missing_values), Missings = missing_values, row.names = NULL)
print(missing_df)
# No hay missing values

   Variable Missings
1  rownames        0
2      wage        0
3     lwage        0
4       sex        0
5       shs        0
6       hsg        0
7       scl        0
8       clg        0
9        ad        0
10       mw        0
11       so        0
12       we        0
13       ne        0
14     exp1        0
15     exp2        0
16     exp3        0
17     exp4        0
18      occ        0
19     occ2        0
20      ind        0
21     ind2        0


### 3. Report descriptive statistics of the variables (mean, standard deviation, percentiles, etc.). Interpret your results.

In [169]:
summary_data <- sapply(data, summary)
summary_df <- as.data.frame(t(summary_data))
print(summary_df)
dim(data)

               Min.     1st Qu.       Median         Mean      3rd Qu.
rownames  10.000000 7132.500000 15260.000000 1.563635e+04 24337.750000
wage       3.021978   13.461538    19.230769 2.341041e+01    27.777778
lwage      1.105912    2.599837     2.956512 2.970787e+00     3.324236
sex        0.000000    0.000000     0.000000 4.444660e-01     1.000000
shs        0.000000    0.000000     0.000000 2.330097e-02     0.000000
hsg        0.000000    0.000000     0.000000 2.438835e-01     0.000000
scl        0.000000    0.000000     0.000000 2.780583e-01     1.000000
clg        0.000000    0.000000     0.000000 3.176699e-01     1.000000
ad         0.000000    0.000000     0.000000 1.370874e-01     0.000000
mw         0.000000    0.000000     0.000000 2.596117e-01     1.000000
so         0.000000    0.000000     0.000000 2.965049e-01     1.000000
we         0.000000    0.000000     0.000000 2.161165e-01     0.000000
ne         0.000000    0.000000     0.000000 2.277670e-01     0.000000
exp1  

[1] 5150   21

## Interpretation
#### 1. Wage: The average wage is 23.41, with significant variability (standard deviation of 21.00), indicating notable income disparity. The wages are heavily skewed, as the maximum wage is 528.85, suggesting some high outliers in the data.
#### 2. Log Wage (lwage): The log-transformed wages have an average of 2.97, with less variability (standard deviation of 0.57). This transformation helps normalize the wage distribution, making it easier to analyze, especially in reducing the effect of extreme values.
#### 3. Sex: About 44% of the sample represents females, which shows a relatively balanced gender distribution, though slightly more males are present.
#### 4. Some High School (shs): Only 2.3% of individuals have not completed high school, which suggests that a small portion of the population has very low educational attainment.
#### 5. High School Graduate (hsg): Around 24% of the sample has graduated from high school. This shows a significant proportion of the population has basic secondary education.
#### 6. Some College (scl): Approximately 28% have attended some college without completing a degree, indicating that a notable portion of the population has started but not finished higher education.
#### 7. College Graduate (clg): About 32% of the sample holds a college degree, reflecting a significant portion of individuals with higher education.
#### 8. Advanced Degree (ad): Roughly 13.7% of the population has completed an advanced degree, showing a smaller, more specialized group with postgraduate qualifications.
#### 9. Midwest (mw): 25.9% of the sample resides in the Midwest, representing a moderate portion of the population in this region.
#### 10. South (so): Around 29.7% live in the South, making it the most populated region in this sample.
#### 11. West (we): 21.6% of individuals are from the West, indicating a lower but still significant portion residing in this area.
#### 12. Northeast (ne): About 22.8% of the population is located in the Northeast, representing a comparable proportion to other regions.
#### 13. Experience 1 (exp1): The average work experience is 13.76 years, with a standard deviation of 10.61. This shows a wide range of experience levels in the dataset, with most individuals falling between 5 and 21 years of experience.
#### 14. Experience 2 (exp2): This transformed variable has a mean of 3.02 and is skewed toward lower values (median of 1). This suggests that most workers have lower values for this variable, with a few outliers.
#### 15. Experience 3 (exp3): The average value is 8.24, but the high standard deviation (14.49) indicates substantial variability. Only 25% of the sample has values above 9.26, but extreme outliers exist, as shown by the maximum of 103.82.
#### 16. Experience 4 (exp4): This variable has the largest variability, with a mean of 25.12 and a standard deviation of 53.53. Most individuals have lower values for this variable (the 75th percentile is 19.45), but there are extreme outliers with values as high as 487.97.

### 4. How many women with a college graduate degree (clg) or above have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result.

In [170]:
# Filtrar las mujeres con título universitario o superior cuyo salario está en el 25% superior
richest <- quantile(data$wage, 0.75)
women <- subset(data, sex == 1 & (clg == 1 | ad == 1))
women_richest <- subset(women, wage >= richest)
print(women_richest)

# Contar cuántas mujeres cumplen con las condiciones
num_women_richest <- nrow(women_richest)
cat('Numero de mujeres con titulo universitario o superior que pertenecen al 25% mas rico:', num_women_richest, '\n')

# A tibble: 419 x 21
   rownames  wage lwage   sex   shs   hsg   scl   clg    ad    mw    so    we
      <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1       19  28.8  3.36     1     0     0     0     1     0     0     0     0
 2      191  42.3  3.74     1     0     0     0     0     1     0     0     0
 3      232  41.2  3.72     1     0     0     0     1     0     0     0     0
 4      319 100    4.61     1     0     0     0     0     1     0     0     0
 5      563  33.7  3.52     1     0     0     0     1     0     0     0     0
 6      842  36.1  3.59     1     0     0     0     1     0     0     0     0
 7      865  31.2  3.44     1     0     0     0     1     0     0     0     0
 8      892  28.8  3.36     1     0     0     0     1     0     0     0     0
 9     1052  48.1  3.87     1     0     0     0     1     0     0     0     0
10     1076  34.6  3.54     1     0     0     0     1     0     0     0     0
# i 409 more rows
# i 9 more variables: ne 

### 5. How many men with a high school graduate degree (hsg) or below have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result.

In [171]:
# Filtrar los hombres con "some highschool" o "highschool graduate" cuyo salario está en el 25% superior
richest <- quantile(data$wage, 0.75)
men <- subset(data, sex == 0 & (shs == 1 | hsg == 1))
men_richest <- subset(men, wage >= richest)
print(men_richest)

# Contar cuántos hombres cumplen con las condiciones
num_men_richest  <- nrow(men_richest)
cat('Número de hombres que se graduaron de highschool o con un grado menor que pertenecen al 25% más rico:', num_men_richest , '\n')

# A tibble: 118 x 21
   rownames  wage lwage   sex   shs   hsg   scl   clg    ad    mw    so    we
      <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1      113  27.9  3.33     0     0     1     0     0     0     0     0     0
 2      276  28.8  3.36     0     0     1     0     0     0     0     0     0
 3      467  28.8  3.36     0     0     1     0     0     0     0     0     0
 4      858  28.8  3.36     0     0     1     0     0     0     0     0     0
 5      876  29.7  3.39     0     0     1     0     0     0     0     0     0
 6     1065  27.8  3.32     0     0     1     0     0     0     0     0     0
 7     1359  35.6  3.57     0     0     1     0     0     0     0     0     0
 8     1870  36.3  3.59     0     0     1     0     0     0     0     0     0
 9     2194  43.3  3.77     0     0     1     0     0     0     0     0     0
10     2214  43.3  3.77     0     0     1     0     0     0     0     0     0
# i 108 more rows
# i 9 more variables: ne 

### 6. Create two dataframes. One containing only the log(wage) and the other containig every variable of the data set but the wage related variables.

In [172]:
salario <- data['lwage']
salario <- as.data.frame(salario)
sin_salario <- data[, !(colnames(data) %in% c('lwage', 'wage'))]
# print(salario)
# print(sin_salario)

# 2. DATA WRANGLING

### 7. Make an array for our  $Y$ variable, which will be the logarithm of wage ($lwage$ column) 

In [173]:
Y <- data$lwage
Y_matrix <- matrix(Y, ncol = 1)
cat("Matriz para la variable Y (logaritmo del salario):\n")
# print(Y_matrix)

Matriz para la variable Y (logaritmo del salario):


### 8. Make three new arrays for our predictors:

#### 8.1. The basic model will have the columns sex hsg scl clg ad so we ne exp1 occ2 ind2. Make sure to convert occ2 and ind2 to dummies and to drop the first dummy value to prevent multicolinearity. This means:

$sex + exp1 + hsg + scl + clg + ad + so + we + ne + dummy(occ2) + dummy(ind2)$ 

In [174]:
# Seleccionar las columnas deseadas
predictors <- sin_salario[, c('sex','exp1', 'hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne', 'occ2', 'ind2')]

# Convertir 'occ2' y 'ind2' en dummies sin incluir la primera categoría
predictors <- dummy_cols(predictors, select_columns = c('occ2', 'ind2'), remove_first_dummy = TRUE)

predictors <- predictors[, !colnames(predictors) %in% c('occ2', 'ind2')]

print(colnames(predictors))
print(dim(predictors))

# Convertir el dataframe a una matriz
predictors_matrix <- as.matrix(predictors)

 [1] "sex"     "exp1"    "hsg"     "scl"     "clg"     "ad"      "so"     
 [8] "we"      "ne"      "occ2_2"  "occ2_3"  "occ2_4"  "occ2_5"  "occ2_6" 
[15] "occ2_7"  "occ2_8"  "occ2_9"  "occ2_10" "occ2_11" "occ2_12" "occ2_13"
[22] "occ2_14" "occ2_15" "occ2_16" "occ2_17" "occ2_18" "occ2_19" "occ2_20"
[29] "occ2_21" "occ2_22" "ind2_3"  "ind2_4"  "ind2_5"  "ind2_6"  "ind2_7" 
[36] "ind2_8"  "ind2_9"  "ind2_10" "ind2_11" "ind2_12" "ind2_13" "ind2_14"
[43] "ind2_15" "ind2_16" "ind2_17" "ind2_18" "ind2_19" "ind2_20" "ind2_21"
[50] "ind2_22"
[1] 5150   50


### 8.2 The flexible model will have the same columns, and will also include polynomials for experience (exp2 exp3 exp4), as well as the interactions between all experience variables and other variables except for sex. This means:

$sex + exp1 + exp2 + exp3 + exp4 + hsg + scl + clg + ad + so + we + ne + dummy(occ2) + dummy(ind2) +(exp1 + exp2 + exp3 + exp4)$ X $(hsg + scl + clg + ad + so + we + ne + dummy(occ2) + dummy(ind2))$ 

Hint: you can use a for loop to multiply the desired variables and create the interactions. Some packages might also have ways to easily specify the model variables with strings like R does by default

In [175]:
# Suponiendo que ya tienes cargado el dataframe "predictors" y "data"

# Crear el dataframe predictors_2 con las variables de experiencia adicionales
predictors_2 <- cbind(predictors, data[, c('exp2', 'exp3', 'exp4')])

# Ordenar las columnas en el orden deseado
desired_order <- c('sex', 'exp1','exp2', 'exp3', 'exp4','hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne',  
                   'occ2_2', 'occ2_3', 'occ2_4', 'occ2_5', 'occ2_6', 'occ2_7', 'occ2_8', 'occ2_9',
                   'occ2_10', 'occ2_11', 'occ2_12', 'occ2_13', 'occ2_14', 'occ2_15', 'occ2_16',
                   'occ2_17', 'occ2_18', 'occ2_19', 'occ2_20', 'occ2_21', 'occ2_22',
                   'ind2_3', 'ind2_4', 'ind2_5', 'ind2_6', 'ind2_7', 'ind2_8', 'ind2_9', 'ind2_10',
                   'ind2_11', 'ind2_12', 'ind2_13', 'ind2_14', 'ind2_15', 'ind2_16', 'ind2_17',
                   'ind2_18', 'ind2_19', 'ind2_20', 'ind2_21', 'ind2_22')

predictors_2 <- predictors_2[, desired_order]
predictors2_matrix <- as.matrix(predictors_2)

# INTERACCIONES
experience_vars <- c('exp1', 'exp2', 'exp3', 'exp4')
other_vars <- setdiff(colnames(predictors), c('sex', 'exp1'))  # exp1 ya está en array_flex1_model

# Crear lista para almacenar las interacciones
interactions <- list()

# Bucle para crear las interacciones
for (exp_var in experience_vars) {
  for (other_var in other_vars) {
    interaction_var <- predictors_2[, exp_var] * predictors_2[, other_var]
    interactions[[paste(exp_var, other_var, sep = "_")]] <- interaction_var
  }
}

# Obtener matriz final
matrix_interactions <- do.call(cbind, interactions)
predictors2_matrix <- cbind(predictors2_matrix, matrix_interactions)

print(colnames(predictors2_matrix))
print(dim(predictors2_matrix))


  [1] "sex"          "exp1"         "exp2"         "exp3"         "exp4"        
  [6] "hsg"          "scl"          "clg"          "ad"           "so"          
 [11] "we"           "ne"           "occ2_2"       "occ2_3"       "occ2_4"      
 [16] "occ2_5"       "occ2_6"       "occ2_7"       "occ2_8"       "occ2_9"      
 [21] "occ2_10"      "occ2_11"      "occ2_12"      "occ2_13"      "occ2_14"     
 [26] "occ2_15"      "occ2_16"      "occ2_17"      "occ2_18"      "occ2_19"     
 [31] "occ2_20"      "occ2_21"      "occ2_22"      "ind2_3"       "ind2_4"      
 [36] "ind2_5"       "ind2_6"       "ind2_7"       "ind2_8"       "ind2_9"      
 [41] "ind2_10"      "ind2_11"      "ind2_12"      "ind2_13"      "ind2_14"     
 [46] "ind2_15"      "ind2_16"      "ind2_17"      "ind2_18"      "ind2_19"     
 [51] "ind2_20"      "ind2_21"      "ind2_22"      "exp1_hsg"     "exp1_scl"    
 [56] "exp1_clg"     "exp1_ad"      "exp1_so"      "exp1_we"      "exp1_ne"     
 [61] "exp1_occ2_2"  "exp1_o

### 8.3 The extra-flexible model will include all two-way interactions between variables, except for sex. This means:

$sex + (exp1 + exp2 + exp3 + exp4 + hsg + scl + clg + ad + so + we + ne + dummy(occ2) + dummy(ind2))^2$ 

Hint: If you use a for loop here, you might create several variables identical to others. You can make another procedure to get rid of duplicates

In [176]:
# Variables
predictors_3 <- predictors_2
predictors3_matrix <- as.matrix(predictors_3)

# Excluir la variable 'sex'
extra_flexible_vars <- colnames(predictors_3)[colnames(predictors_3) != 'sex']

# Crear una lista para almacenar las interacciones
interactions <- list()

# Crear las interacciones
for (i in seq_along(extra_flexible_vars)) {
  var1 <- extra_flexible_vars[i]
  for (j in (i+1):length(extra_flexible_vars)) {
    var2 <- extra_flexible_vars[j]
    # Crear la variable de interacción
    interaction_var <- predictors_3[[var1]] * predictors_3[[var2]]
    # Añadir la variable de interacción a la lista
    interactions[[paste(var1, var2, sep = "_")]] <- interaction_var
  }
}

interactions_matrix <- do.call(cbind, interactions)

# Añadir la variable 'sex' (valores) al principio de la matriz de interacciones
predictors3_matrix <- cbind(predictors_3[["sex"]], as.matrix(predictors_3[, extra_flexible_vars]), interactions_matrix)

# Convertir a un dataframe para poder modificar el nombre de la primera columna que aparece ""
predictors3_df <- as.data.frame(predictors3_matrix)

# Asignar nombre a la variable sex antes extraida
colnames(predictors3_df) <- c("sex", colnames(predictors_3[, extra_flexible_vars]), colnames(interactions_matrix))

# Convertir de nuevo a matriz 
predictors3_matrix <- as.matrix(predictors3_df)

dim(predictors3_matrix)
print(colnames(predictors3_matrix))


[1] 5150 1380

   [1] "sex"             "exp1"            "exp2"            "exp3"           
   [5] "exp4"            "hsg"             "scl"             "clg"            
   [9] "ad"              "so"              "we"              "ne"             
  [13] "occ2_2"          "occ2_3"          "occ2_4"          "occ2_5"         
  [17] "occ2_6"          "occ2_7"          "occ2_8"          "occ2_9"         
  [21] "occ2_10"         "occ2_11"         "occ2_12"         "occ2_13"        
  [25] "occ2_14"         "occ2_15"         "occ2_16"         "occ2_17"        
  [29] "occ2_18"         "occ2_19"         "occ2_20"         "occ2_21"        
  [33] "occ2_22"         "ind2_3"          "ind2_4"          "ind2_5"         
  [37] "ind2_6"          "ind2_7"          "ind2_8"          "ind2_9"         
  [41] "ind2_10"         "ind2_11"         "ind2_12"         "ind2_13"        
  [45] "ind2_14"         "ind2_15"         "ind2_16"         "ind2_17"        
  [49] "ind2_18"         "ind2_19"         "ind2_20"

# 3. LINEAR REGRESSIONS

9. was omitted

### 10. Split each of the dataframes created (basic, flexible and extra-flexible models) into a training sample (80% of the data) and a test sample. Use the normalized data for this. Hint: You do not need to normalize the data for dummy variables.

In [177]:
# Formamos el conjunto de datos para cada modelo
data_1 <- data.frame(salario,predictors_matrix)
data_2 <- data.frame(salario,predictors2_matrix)
data_3 <- data.frame(salario,predictors3_matrix)
print(dim(data_1))
print(dim(data_2))
print(dim(data_3))

[1] 5150   51
[1] 5150  246
[1] 5150 1381


In [178]:
# No se pudo hacer de esta manera
# sample_1 <- sample.split(data_1$salario, SplitRatio = 0.8, group = NULL) 
# train_1 <- subset(data_1, sample_1 == TRUE)
# test_1 <- subset(data_1, sample_1 == FALSE)

# sample_2 <- sample.split(data_2$salario, SplitRatio = 0.8, group = NULL)
# train_2 <- subset(data_2, sample_2 == TRUE)
# test_2 <- subset(data_2, sample_2 == FALSE)

# sample_3 <- sample.split(data_3$salario, SplitRatio = 0.8, group = NULL)
# train_3 <- subset(data_3, sample_3 == TRUE)
# test_3 <- subset(data_3, sample_3 == FALSE)

In [179]:
# Hacemos de una forma equivalente
# Crear índice aleatorio para 80% de los datos de entrenamiento
train_index_1 <- sample(1:nrow(data_1), size = 0.8 * nrow(data_1))
train_1 <- data_1[train_index_1, ]  # Datos de entrenamiento
test_1 <- data_1[-train_index_1, ]  # Datos de prueba

train_index_2 <- sample(1:nrow(data_2), size = 0.8 * nrow(data_2))
train_2 <- data_2[train_index_2, ]
test_2 <- data_2[-train_index_2, ]

train_index_3 <- sample(1:nrow(data_3), size = 0.8 * nrow(data_3))
train_3 <- data_3[train_index_3, ]
test_3 <- data_3[-train_index_3, ]

print(dim(train_1))
print(dim(test_1))
print(dim(train_2))
print(dim(test_2))
print(dim(train_3))
print(dim(test_3))

[1] 4120   51
[1] 1030   51
[1] 4120  246
[1] 1030  246
[1] 4120 1381
[1] 1030 1381


### 11. Estimate all three models
### - basic
### - flexible
### - extra-flexible model

In [180]:
# Basic Model: Model 1
model_1 <- lm(lwage~., data = data_1) # ponemos como lwage porque es el nombre de la variable
model_1



Call:
lm(formula = lwage ~ ., data = data_1)

Coefficients:
(Intercept)          sex         exp1          hsg          scl          clg  
   3.101896    -0.072857     0.008568     0.088461     0.180805     0.410582  
         ad           so           we           ne       occ2_2       occ2_3  
   0.592798    -0.006913     0.044790     0.027541    -0.076472    -0.034678  
     occ2_4       occ2_5       occ2_6       occ2_7       occ2_8       occ2_9  
  -0.096202    -0.187915    -0.414933    -0.045987    -0.377847    -0.215752  
    occ2_10      occ2_11      occ2_12      occ2_13      occ2_14      occ2_15  
  -0.010623    -0.455834    -0.307589    -0.361440    -0.499495    -0.464482  
    occ2_16      occ2_17      occ2_18      occ2_19      occ2_20      occ2_21  
  -0.233715    -0.412588    -0.340418    -0.241480    -0.212628    -0.288413  
    occ2_22       ind2_3       ind2_4       ind2_5       ind2_6       ind2_7  
  -0.422394    -0.116836    -0.244493    -0.273533    -0.249368    -0.

In [181]:
# Flexible Model: Model 2
model_2 <- lm(lwage~., data = data_2)
model_2


Call:
lm(formula = lwage ~ ., data = data_2)

Coefficients:
 (Intercept)           sex          exp1          exp2          exp3  
    3.847635     -0.069553     -0.287716      3.737361     -1.650175  
        exp4           hsg           scl           clg            ad  
    0.226921     -0.405593     -0.168749      0.082145      0.123309  
          so            we            ne        occ2_2        occ2_3  
   -0.088259     -0.132249     -0.110683      0.161340      0.210151  
      occ2_4        occ2_5        occ2_6        occ2_7        occ2_8  
    0.070857     -0.396008     -0.231061      0.314725     -0.187542  
      occ2_9       occ2_10       occ2_11       occ2_12       occ2_13  
   -0.339027      0.020954     -0.642418     -0.067477     -0.232978  
     occ2_14       occ2_15       occ2_16       occ2_17       occ2_18  
    0.256201     -0.193858     -0.055126     -0.415609     -0.482217  
     occ2_19       occ2_20       occ2_21       occ2_22        ind2_3  
   -0.257941    

In [182]:
# Extra-flexible model: Model 3
model_3 <- lm(lwage~., data = data_3)
model_3


Call:
lm(formula = lwage ~ ., data = data_3)

Coefficients:
    (Intercept)              sex             exp1             exp2  
      2.977e+00       -6.127e-02       -2.043e-01        3.304e+00  
           exp3             exp4              hsg              scl  
     -1.697e+00        1.862e-01       -4.580e-01        9.478e-03  
            clg               ad               so               we  
      7.734e-01        9.480e-01       -9.637e-02        4.382e-01  
             ne           occ2_2           occ2_3           occ2_4  
     -5.137e-01       -5.386e-01        4.728e-02        7.844e-01  
         occ2_5           occ2_6           occ2_7           occ2_8  
     -8.022e-01       -7.751e-01       -9.971e-01       -5.260e-01  
         occ2_9          occ2_10          occ2_11          occ2_12  
     -1.009e+00       -1.659e+00       -9.453e-01        2.169e-02  
        occ2_13          occ2_14          occ2_15          occ2_16  
     -4.136e-01        9.991e-02       -1.

### 13. Report the MSE and the $R^2$ for both samples of each model, and the adjusted $R^2$ for the training sample. Do we see overfitting? Which model has the best out-of-sample performance? For reference, you should report the following:

In [183]:
dim(train_1)
dim(train_2)
dim(train_3)

[1] 4120   51

[1] 4120  246

[1] 4120 1381

In [184]:
# Para la muestra de entrenamiento (ajuste)
model_train_1 <- lm(lwage~., data = train_1)
model_train_2 <- lm(lwage~., data = train_2)
model_train_3 <- lm(lwage~., data = train_3)

y_hat_train_1 <- model_train_1$fitted.values
y_hat_train_2 <- model_train_2$fitted.values
y_hat_train_3 <- model_train_3$fitted.values

y_1_train = train_1$lwage
y_2_train = train_2$lwage
y_3_train = train_3$lwage

# MSE 
mse_train_1 <- mean((y_1_train - y_hat_train_1)^2)
mse_train_2 <- mean((y_2_train - y_hat_train_2)^2)
mse_train_3 <- mean((y_3_train - y_hat_train_3)^2)

# R cuadrado
R_sq_train_1 <- 1 - mse_train_1/mean((y_1_train - mean(y_1_train))^2)
R_sq_train_2 <- 1 - mse_train_2/mean((y_2_train - mean(y_2_train))^2)
R_sq_train_3 <- 1 - mse_train_3/mean((y_3_train - mean(y_3_train))^2)

# R cuadrado ajustado
adj_mse_train_1 = 4120 / (4120-51) * mse_train_1
adjR_sq_train_1 = 1 - adj_mse_train_1 / mean((y_1_train - mean(y_1_train))^2)

adj_mse_train_2 = 4120 / (4120-246) * mse_train_2
adjR_sq_train_2 = 1 - adj_mse_train_2 / mean((y_2_train - mean(y_2_train))^2)

adj_mse_train_3 = 4120 / (4120-1381) * mse_train_3
adjR_sq_train_3 = 1 - adj_mse_train_3 / mean((y_3_train - mean(y_3_train))^2)

In [185]:
# Para la muestra de testeo (performance del modelo)
# Extraemos endógena de la muestra de testeo y entrenamiento
y_1_test = test_1$lwage
y_2_test = test_2$lwage
y_3_test = test_3$lwage

# Agrupamos a los regresores de la muestra de testeo y entrenamiento
test_1_regresors <- test_1[, !names(test_1) %in% "lwage"]
test_2_regresors <- test_2[, !names(test_2) %in% "lwage"]
test_3_regresors <- test_3[, !names(test_3) %in% "lwage"]
 
# Realizamos predicción
y_1_hat_test = predict(model_train_1, test_1_regresors)
y_2_hat_test = predict(model_train_2, test_2_regresors)
y_3_hat_test = predict(model_train_3, test_3_regresors)

# MSE
mse_test_1 <- mean((y_1_test-y_1_hat_test)^2)
mse_test_2 <- mean((y_2_test-y_2_hat_test)^2)
mse_test_3 <- mean((y_3_test-y_3_hat_test)^2)

# R cuadrado
R_sq_test_1 <- 1 - mse_test_1 / mean((y_1_test - mean(y_1_test))^2)
R_sq_test_2 <- 1 - mse_test_2 / mean((y_2_test - mean(y_2_test))^2)
R_sq_test_3 <- 1 - mse_test_3 / mean((y_3_test - mean(y_3_test))^2)

Warning message in predict.lm(model_train_2, test_2_regresors):
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(model_train_3, test_3_regresors):
"prediction from a rank-deficient fit may be misleading"


In [186]:
print(paste0("El MSE de la muestra de testeo del modelo 1 es ", mse_test_1))
print(paste0("El MSE de la muestra de testeo del modelo 2 es ", mse_test_2))
print(paste0("El MSE de la muestra de testeo del modelo 3 es ", mse_test_3))
print(paste0("El MSE de la muestra de entrenamiento del modelo 1 es ", mse_train_1))
print(paste0("El MSE de la muestra de entrenamiento del modelo 2 es ", mse_train_2))
print(paste0("El MSE de la muestra de entrenamiento del modelo 3 es ", mse_train_3))
print(paste0("El R cuadrado de la muestra de testeo del modelo 1 es ", R_sq_test_1))
print(paste0("El R cuadrado de la muestra de testeo del modelo 2 es ", R_sq_test_2))
print(paste0("El R cuadrado de la muestra de testeo del modelo 3 es ", R_sq_test_3))
print(paste0("El R cuadrado de la muestra de entrenamiento del modelo 1 es ", R_sq_train_1))
print(paste0("El R cuadrado de la muestra de entrenamiento del modelo 2 es ", R_sq_train_2))
print(paste0("El R cuadrado de la muestra de entrenamiento del modelo 3 es ", R_sq_train_3))
print(paste0("El R cuadrado ajustado de la muestra de entrenamiento del modelo 1 es ", adjR_sq_train_1))
print(paste0("El R cuadrado ajustado de la muestra de entrenamiento del modelo 2 es ", adjR_sq_train_2))
print(paste0("El R cuadrado ajustado de la muestra de entrenamiento del modelo 3 es ", adjR_sq_train_3))

[1] "El MSE de la muestra de testeo del modelo 1 es 0.244079920865103"
[1] "El MSE de la muestra de testeo del modelo 2 es 0.238011762829535"
[1] "El MSE de la muestra de testeo del modelo 3 es 0.318241456634534"
[1] "El MSE de la muestra de entrenamiento del modelo 1 es 0.220355792416746"
[1] "El MSE de la muestra de entrenamiento del modelo 2 es 0.212648223581751"
[1] "El MSE de la muestra de entrenamiento del modelo 3 es 0.172738692162447"
[1] "El R cuadrado de la muestra de testeo del modelo 1 es 0.313704840854209"
[1] "El R cuadrado de la muestra de testeo del modelo 2 es 0.252707553278393"
[1] "El R cuadrado de la muestra de testeo del modelo 3 es 0.0561378606898287"
[1] "El R cuadrado de la muestra de entrenamiento del modelo 1 es 0.306206362582108"
[1] "El R cuadrado de la muestra de entrenamiento del modelo 2 es 0.349495277848632"
[1] "El R cuadrado de la muestra de entrenamiento del modelo 3 es 0.464040226447198"
[1] "El R cuadrado ajustado de la muestra de entrenamiento del 

### Do we see overfitting?
#### Overfitting is observed in the Extra Flexible Model (model_3), as it has significantly better performance on the training sample compared to the test sample, and a negative $R^2$ on the test sample
### Which model has the best out-of-sample performance?
#### The flexible model demonstrates the best out-of-sample performance, with a relatively high adjusted $R^2$, indicating a good fit while penalizing excessive features. In contrast, the extra-flexible model, despite having a high $R^2$ in training, shows a negative $R^2$ on the test sample, indicating overfitting.